In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score, accuracy_score
from collections import defaultdict
from itertools import combinations

In [2]:
# PROBLEM 1
data = pd.read_csv('diabetes_risk_prediction.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

binary_columns = ['Polyuria', 'Polydipsia', 'sudden weight loss', 'weakness', 
                 'Polyphagia', 'Genital thrush', 'visual blurring', 'Itching',
                 'Irritability', 'delayed healing', 'partial paresis', 
                 'muscle stiffness', 'Alopecia', 'Obesity']

# Convert binary columns to integers
for column in binary_columns:
    data[column] = (data[column] == 'Yes').astype(int)

# Define the label to be predicted
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])

# Convert the label to be predicted to integers
data['class'] = (data['class'] == 'Positive').astype(int)

X = data.drop('class', axis=1).values
y = data['class'].values

def evaluate_model(model, X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': []}
    
    # Iterate over the folds 
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Standardize the data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
        metrics['accuracy'].append(accuracy_score(y_test, y_pred))
        metrics['precision'].append(precision_score(y_test, y_pred))
        metrics['recall'].append(recall_score(y_test, y_pred))
        
    return {k: np.mean(v) for k, v in metrics.items()}

# Define the models
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='rbf', random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Naive Bayes': GaussianNB()
}

results = {}
for name, model in models.items():
    print(f"\nResults for {name}:")
    results[name] = evaluate_model(model, X, y)
    for metric, value in results[name].items():
        print(f"{metric.capitalize()}: {value:.3f}")



Results for Decision Tree:
Accuracy: 0.963
Precision: 0.984
Recall: 0.957

Results for KNN:
Accuracy: 0.935
Precision: 0.971
Recall: 0.923

Results for SVM:
Accuracy: 0.967
Precision: 0.978
Recall: 0.968

Results for Logistic Regression:
Accuracy: 0.919
Precision: 0.945
Recall: 0.923

Results for Naive Bayes:
Accuracy: 0.887
Precision: 0.907
Recall: 0.909


In [3]:
# PROBELM 2
def create_transition_matrix(adjacency_list, n_nodes):
    """
    Create a transition matrix from an adjacency list.
    """

    # Initialize the transition matrix
    M = np.zeros((n_nodes, n_nodes))
    
    # Iterate over each node in the adjacency list
    for i in range(n_nodes):
        out_degree = len(adjacency_list[i])
        # If the node has outgoing edges
        # assign the corresponding probabilities
        if out_degree > 0:
            for j in adjacency_list[i]:
                M[j-1][i] = 1.0 / out_degree
    
    return M

def pagerank(M, num_iterations=100, d=0.85):
    """
    Compute the PageRank values using the power iteration method.
    """

    n = M.shape[0]
    
    pr = np.ones(n) / n
    
    # Iterate until convergence
    for _ in range(num_iterations):
        # Compute the next PageRank values
        pr_next = (1 - d) / n + d * M @ pr
        
        if np.allclose(pr, pr_next):
            break
            
        pr = pr_next
    
    return pr

# Define the adjacency list for the graph
adjacency_list = {
    0: [2, 3, 4],
    1: [4, 5],
    2: [6],
    3: [6, 7],
    4: [7],
    5: [1],
    6: [4, 6]
}

n_nodes = 7

M = create_transition_matrix(adjacency_list, n_nodes)
pagerank_values = pagerank(M)
node_rankings = [(i+1, pr) for i, pr in enumerate(pagerank_values)]
sorted_rankings = sorted(node_rankings, key=lambda x: x[1], reverse=True)

print("\nPageRank values:")
for node, value in node_rankings:
    print(f"Node {node}: {value:.4f}")

print("\nNodes sorted by PageRank (highest to lowest):")
print(" > ".join([f"Node {node}" for node, _ in sorted_rankings]))

print("\nDetailed ranking:")
for rank, (node, value) in enumerate(sorted_rankings, 1):
    print(f"{rank}. Node {node} (PageRank: {value:.4f})")


PageRank values:
Node 1: 0.2180
Node 2: 0.0832
Node 3: 0.0832
Node 4: 0.1809
Node 5: 0.0568
Node 6: 0.2313
Node 7: 0.1466

Nodes sorted by PageRank (highest to lowest):
Node 6 > Node 1 > Node 4 > Node 7 > Node 2 > Node 3 > Node 5

Detailed ranking:
1. Node 6 (PageRank: 0.2313)
2. Node 1 (PageRank: 0.2180)
3. Node 4 (PageRank: 0.1809)
4. Node 7 (PageRank: 0.1466)
5. Node 2 (PageRank: 0.0832)
6. Node 3 (PageRank: 0.0832)
7. Node 5 (PageRank: 0.0568)


In [4]:
# PROBLEM 3
def load_transactions(data):
    """
    Load transactions from a string
    """
    transactions = []
    for line in data.strip().split('\n'):
        transaction = set(map(int, line.strip().split()))
        transactions.append(transaction)
    return transactions

def apriori(transactions, min_support):
    """
    Find frequent itemsets using the Apriori algorithm
    """
    # Count items
    item_counts = defaultdict(int)
    total_transactions = len(transactions)
    
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1
    
    # Find frequent items
    frequent_items = {
        frozenset([item]): count/total_transactions 
        for item, count in item_counts.items() 
        if count/total_transactions >= min_support
    }
    
    # Find frequent itemsets
    all_frequent_itemsets = {1: frequent_items} # Store frequent itemsets for each size
    current_itemsets = frequent_items
    
    k = 2 # Current itemset size
    # Continue until no frequent itemsets are found or until the maximum transaction length is reached
    while current_itemsets and k <= max(len(t) for t in transactions):
        candidates = set()
        for item1, item2 in combinations(current_itemsets.keys(), 2):
            union = item1.union(item2)
            if len(union) == k:
                candidates.add(union)
        
        # Count candidate itemsets
        itemset_counts = defaultdict(int)
        for transaction in transactions:
            for candidate in candidates:
                if candidate.issubset(transaction):
                    itemset_counts[candidate] += 1
        
        # Find frequent itemsets
        current_itemsets = {
            itemset: count/total_transactions
            for itemset, count in itemset_counts.items()
            if count/total_transactions >= min_support
        }
        
        if current_itemsets:
            all_frequent_itemsets[k] = current_itemsets # Store frequent itemsets for this size
            
        k += 1
    
    return all_frequent_itemsets

def find_max_support_size4(transactions, precision=0.001):
    """
    Find the maximum support value for a size-4 itemset
    """
    left = 0.0
    right = 1.0
    max_support = 0.0
    
    # Binary search for the maximum support value
    while right - left >= precision:
        mid = (left + right) / 2
        frequent_sets = apriori(transactions, mid)
        
        has_size4 = any(k >= 4 for k in frequent_sets.keys())
        
        if has_size4:
            max_support = mid
            left = mid + precision
        else:
            right = mid
    
    return max_support

data = open('kosarak.dat').read()
transactions = load_transactions(data)

print("Part a: Frequent itemsets with minSupport = 0.15")
frequent_sets = apriori(transactions, 0.15)
for k, itemsets in frequent_sets.items():
    print(f"\nFrequent {k}-itemsets:")
    for itemset, support in itemsets.items():
        print(f"Items: {sorted(itemset)}, Support: {support:.3f}")

max_support = find_max_support_size4(transactions)
print(f"\nPart b: Maximum support value for size-4 itemset: {max_support:.3f}")

Part a: Frequent itemsets with minSupport = 0.15

Frequent 1-itemsets:
Items: [1], Support: 0.200
Items: [3], Support: 0.455
Items: [6], Support: 0.607
Items: [11], Support: 0.368

Frequent 2-itemsets:
Items: [6, 11], Support: 0.327
Items: [3, 6], Support: 0.268
Items: [3, 11], Support: 0.163

Part b: Maximum support value for size-4 itemset: 0.050


In [5]:
# PROBLEM 4
def normalize(vector):
    """
    Normalize a vector to have a unit norm.
    """
    norm = sum(x * x for x in vector) ** 0.5
    return [x / norm if norm > 0 else 0 for x in vector]

def hits(adjacency_matrix, tolerance=0.0001, max_iterations=100):
    """
    Compute the HITS algorithm on a given adjacency matrix.
    """
    n = len(adjacency_matrix)
    hubs = [1.0] * n
    authorities = [1.0] * n
    
    steps = 0
    for _ in range(max_iterations):
        steps += 1
        
        # Store previous values for convergence check
        prev_hubs = hubs.copy()
        prev_authorities = authorities.copy()
        
        # Update authorities
        new_authorities = [0.0] * n
        for i in range(n):
            for j in range(n):
                if adjacency_matrix[j][i] == 1:
                    new_authorities[i] += hubs[j]
        
        # Update hubs
        new_hubs = [0.0] * n
        for i in range(n):
            for j in range(n):
                if adjacency_matrix[i][j] == 1:
                    new_hubs[i] += new_authorities[j]
        
        hubs = normalize(new_hubs)
        authorities = normalize(new_authorities)
        
        # Check for convergence
        hub_diff = sum((h1 - h2)**2 for h1, h2 in zip(hubs, prev_hubs))
        auth_diff = sum((a1 - a2)**2 for a1, a2 in zip(authorities, prev_authorities))
        
        if hub_diff < tolerance and auth_diff < tolerance:
            break
    
    return hubs, authorities, steps

# Define three different graphs
# Graph 1: Simple chain (4 nodes)
graph1 = [
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 0]
]

# Graph 2: Star topology (5 nodes)
graph2 = [
    [0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
]

# Graph 3: Cycle with central node (5 nodes)
graph3 = [
    [0, 1, 0, 0, 1],
    [0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1],
    [1, 0, 0, 0, 1],
    [1, 1, 1, 1, 0]
]

tolerance = 0.0001

for i, graph in enumerate([graph1, graph2, graph3], 1):
    hubs, authorities, steps = hits(graph, tolerance)
    
    print(f"\nGraph {i} Results:")
    print("Hub scores:", [f"{x:.4f}" for x in hubs])
    print("Authority scores:", [f"{x:.4f}" for x in authorities])
    print(f"Convergence achieved in {steps} steps")


Graph 1 Results:
Hub scores: ['0.5774', '0.5774', '0.5774', '0.0000']
Authority scores: ['0.0000', '0.5774', '0.5774', '0.5774']
Convergence achieved in 2 steps

Graph 2 Results:
Hub scores: ['1.0000', '0.0000', '0.0000', '0.0000', '0.0000']
Authority scores: ['0.0000', '0.5000', '0.5000', '0.5000', '0.5000']
Convergence achieved in 2 steps

Graph 3 Results:
Hub scores: ['0.3945', '0.3945', '0.3945', '0.3945', '0.6143']
Authority scores: ['0.3934', '0.3934', '0.3934', '0.3934', '0.6173']
Convergence achieved in 5 steps
